In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, recall_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [15]:
import pandas as pd

In [16]:
# Import pre-processed data
dftrain = pd.read_csv("train_final.csv")
dfvalid = pd.read_csv("validation_final.csv")
dftest = pd.read_csv("test_final.csv")

In [19]:
# Split into training and testing x,y
ytrain = dftrain['Overall_Experience']
Xtrain = dftrain.drop(columns = "Overall_Experience")
yvalid = dfvalid['Overall_Experience']
Xvalid = dfvalid.drop(columns = "Overall_Experience")
Xtrain.shape, ytrain.shape, Xvalid.shape, yvalid.shape

((75503, 23), (75503,), (18876, 23), (18876,))

In [22]:
trainfinal = pd.concat([dftrain, dfvalid], axis = 0)
ytrain_final = trainfinal['Overall_Experience']
Xtrain_final = trainfinal.drop(columns = "Overall_Experience")

In [24]:
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer,mean_squared_error, r2_score, mean_absolute_error

In [26]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(Xtrain_final, ytrain_final)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [ ]:
rf_random.best_params_

In [ ]:
prediction_tuned = rf_random.predict()

In [ ]:
importances = rf.feature_importances_

indices = np.argsort(importances)

plt.figure(figsize = (10, 10))

plt.title('Feature Importances')

plt.barh(range(len(indices)), importances[indices], color = 'violet', align = 'center')

plt.yticks(range(len(indices)), [features[i] for i in indices])

plt.xlabel('Relative Importance')

plt.show()

In [ ]:
ID_data = pd.read_csv("Traveldata_test.csv")
submission_df = pd.DataFrame()
submission_df['ID'] = ID_data['ID']
submission_df['Overall_Experience'] = prediction_tuned
submission_df['Overall_Experience'] = submission_df['Overall_Experience'].round(0).astype(int)
submission_df.to_csv("submission_4.csv", index=False)